In [1]:
import os
import json
from pathlib import Path
import pandas as pd
import numpy as np
import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report
)


def _save_json(p, obj):
    p.parent.mkdir(parents=True, exist_ok=True)
    with open(p, "w") as f:
        json.dump(obj, f, indent=2)


def train_from_npy(
    train_npy_path,
    model_out_dir,
    train_labels_npy_path,
    random_seed=42,
    mem_limit_bytes=100_000_000_000,
    use_mmap=False,
    classifier='LogisticRegression'
):
    train_npy_path = Path(train_npy_path)
    if not train_npy_path.exists():
        raise FileNotFoundError(f"Train npy not found: {train_npy_path}")

    model_out_dir = Path(model_out_dir)
    model_out_dir.mkdir(parents=True, exist_ok=True)

    X_train = np.load(str(train_npy_path), mmap_mode="r" if use_mmap else None)
    if X_train.ndim != 2:
        raise RuntimeError(f"train array must be 2D. got shape={X_train.shape}")

    labels_path = Path(train_labels_npy_path)
    if not labels_path.exists():
        raise FileNotFoundError(f"Provided train_labels_npy_path does not exist: {labels_path}")
    y_train = np.load(str(labels_path), mmap_mode="r" if use_mmap else None)
    if y_train.shape[0] != X_train.shape[0]:
        raise RuntimeError(f"train labels length ({y_train.shape[0]}) != train rows ({X_train.shape[0]})")

    y_train = y_train.astype(np.int64)
    unique = np.unique(y_train)
    if unique.size != 2 or not np.array_equal(np.sort(unique), np.array([0, 1])):
        raise RuntimeError(f"train labels must contain exactly two classes 0 and 1. found: {unique}")
    y_train = y_train.astype(np.uint8)

    estimated_bytes = getattr(X_train, "nbytes", X_train.size * X_train.itemsize)
    if estimated_bytes > mem_limit_bytes:
        raise MemoryError(
            f"Estimated train bytes {estimated_bytes} > mem_limit_bytes {mem_limit_bytes}. "
            "Either increase mem_limit_bytes, reduce dataset size, or use use_mmap=True and implement chunked/out-of-core training."
        )

    scaler = MinMaxScaler()

    if classifier == 'LogisticRegression':
        print ('Using LogisticRegression as classifier')
        clf = LogisticRegression(random_state=random_seed, n_jobs=-1, verbose=1)
    elif classifier == 'RandomForest':
        print ('Using RandomForest as classifier')
        clf = RandomForestClassifier(random_state=random_seed, n_jobs=-1, verbose=1)
    else:
        print ('Using SVC as classifier')
        clf = SVC(random_state=random_seed, verbose=0)

    pipeline = Pipeline([("scaler", scaler), ("clf", clf)])

    print(f"Fitting pipeline")
    pipeline.fit(X_train, y_train)

    try:
        y_pred_train, y_score_train = _predict_and_score(pipeline, None, None, X_train)
        train_metrics = _compute_metrics(y_train, y_pred_train, y_score_train)

        print("TRAIN Eval results:")
        print(f"  acc={train_metrics['accuracy']:.4f}  prec={train_metrics['precision']:.4f}  "
              f"recall={train_metrics['recall']:.4f}  f1={train_metrics['f1']:.4f}")
        if train_metrics.get("roc_auc") is not None:
            print(f"  roc_auc={train_metrics['roc_auc']:.4f}")
        print("  confusion_matrix:", train_metrics["confusion_matrix"])
    except Exception as e:

        print("Warning: failed to compute train metrics:", e)
        train_metrics = None

    pipeline_path = model_out_dir / "pipeline.joblib"
    scaler_path = model_out_dir / "scaler.joblib"
    clf_path = model_out_dir / "clf.joblib"
    meta_out = model_out_dir / "train_meta.json"

    joblib.dump(pipeline, pipeline_path)
    joblib.dump(pipeline.named_steps["scaler"], scaler_path)
    joblib.dump(pipeline.named_steps["clf"], clf_path)

    meta_obj = {
        "train_npy": str(train_npy_path),
        "train_labels_npy": str(train_labels_npy_path),
        "train_rows": int(X_train.shape[0]),
        "C": None,
        "random_seed": int(random_seed),
        "use_mmap": bool(use_mmap),
        "train_metrics": train_metrics,
    }
    _save_json(meta_out, meta_obj)

    print("Saved pipeline ->", pipeline_path)
    print("Saved scaler ->", scaler_path)
    print("Saved classifier ->", clf_path)
    return str(model_out_dir)

def _load_model(model_out_dir):
    model_out_dir = Path(model_out_dir)
    pipeline_path = model_out_dir / "pipeline.joblib"
    scaler_path = model_out_dir / "scaler.joblib"
    clf_path = model_out_dir / "clf.joblib"

    pipeline = None
    scaler = None
    clf = None

    if pipeline_path.exists():
        pipeline = joblib.load(pipeline_path)
    else:
        if scaler_path.exists():
            scaler = joblib.load(scaler_path)
        if clf_path.exists():
            clf = joblib.load(clf_path)
        if scaler is None or clf is None:
            raise FileNotFoundError("Could not find pipeline or scaler+clf in model_out_dir.")
    return pipeline, scaler, clf


def _compute_metrics(y_true, y_pred, y_score):
    metrics = {}
    metrics["accuracy"] = float(accuracy_score(y_true, y_pred))
    metrics["precision"] = float(precision_score(y_true, y_pred, zero_division=0))
    metrics["recall"] = float(recall_score(y_true, y_pred, zero_division=0))
    metrics["f1"] = float(f1_score(y_true, y_pred, zero_division=0))
    try:
        if y_score is not None and len(np.unique(y_true)) == 2:
            metrics["roc_auc"] = float(roc_auc_score(y_true, y_score))
        else:
            metrics["roc_auc"] = None
    except Exception:
        metrics["roc_auc"] = None
    metrics["confusion_matrix"] = confusion_matrix(y_true, y_pred).tolist()
    metrics["classification_report"] = classification_report(y_true, y_pred, zero_division=0, output_dict=True)
    return metrics


def _predict_and_score(pipeline, scaler, clf, X_test):
    y_score = None
    if pipeline is not None:
        y_pred = pipeline.predict(X_test)
        if hasattr(pipeline, "predict_proba"):
            try:
                y_score = pipeline.predict_proba(X_test)[:, 1]
            except Exception:
                y_score = None
        else:
            try:
                clf_ = pipeline.named_steps["clf"]
                scaler_ = pipeline.named_steps["scaler"]
                if hasattr(clf_, "decision_function"):
                    y_score = clf_.decision_function(scaler_.transform(X_test))
            except Exception:
                y_score = None
    else:
        X_t = scaler.transform(X_test)
        y_pred = clf.predict(X_t)
        if hasattr(clf, "predict_proba"):
            y_score = clf.predict_proba(X_t)[:, 1]
        elif hasattr(clf, "decision_function"):
            y_score = clf.decision_function(X_t)
    return y_pred, y_score


def eval_val_from_npy(
    val_npy_path,
    val_label_path,
    model_out_dir="./models",
    return_predictions=False,
    use_mmap=False,
    csv_path=None, 
):
    val_npy_path = Path(val_npy_path)
    val_label_path = Path(val_label_path)
    if not val_npy_path.exists():
        raise FileNotFoundError(f"val_npy not found: {val_npy_path}")

    X_test = np.load(str(val_npy_path), mmap_mode="r" if use_mmap else None).astype(np.float32)
    y_test = np.load(str(val_label_path), mmap_mode="r" if use_mmap else None).astype(np.uint8)
    
    if X_test.shape[0] != y_test.shape[0]:
        raise RuntimeError(f"val X rows ({X_test.shape[0]}) != val labels length ({y_test.shape[0]})")

    pipeline, scaler, clf = _load_model(model_out_dir)

    y_pred, y_score = _predict_and_score(pipeline, scaler, clf, X_test)

    metrics = _compute_metrics(y_test, y_pred, y_score)

    print("VAL Eval results:")
    print(f"  acc={metrics['accuracy']:.4f}  prec={metrics['precision']:.4f}  recall={metrics['recall']:.4f}  f1={metrics['f1']:.4f}")
    if metrics["roc_auc"] is not None:
        print(f"  roc_auc={metrics['roc_auc']:.4f}")
    print("  confusion_matrix:", metrics["confusion_matrix"])

    if csv_path is not None:
        csv_path = Path(csv_path)
        csv_path.parent.mkdir(parents=True, exist_ok=True)

        metrics_df = pd.DataFrame([metrics])
        metrics_df.to_csv(csv_path, index=False)

        if return_predictions:
            preds_path = csv_path.with_name(csv_path.stem + "_predictions.csv")
            preds_df = pd.DataFrame({
                "y_true": y_test,
                "y_pred": y_pred,
                "y_score": y_score
            })
            preds_df.to_csv(preds_path, index=False)
            print(f"Predictions saved to: {preds_path}")

        print(f"Metrics saved to: {csv_path}")

    return {
        "metrics": metrics,
        "y_true": y_test if return_predictions else None,
        "y_pred": y_pred if return_predictions else None,
        "y_score": y_score if return_predictions else None,
    }


def eval_test_from_npy(
    test_clean_npy_path,
    test_adv_npy_path=None,
    model_out_dir="./models",
    test_labels_npy_path=None,
    return_predictions=False,
    use_mmap=False,
    csv_path=None,  
):
    test_clean_npy_path = Path(test_clean_npy_path)
    if not test_clean_npy_path.exists():
        raise FileNotFoundError(f"test_clean npy not found: {test_clean_npy_path}")

    if test_adv_npy_path is None:
        if test_labels_npy_path is None:
            raise ValueError("Single-file mode requires test_labels_npy_path.")
        X_test = np.load(str(test_clean_npy_path), mmap_mode="r" if use_mmap else None).astype(np.float32)
        y_test = np.load(str(test_labels_npy_path), mmap_mode="r" if use_mmap else None).astype(np.uint8)
        if X_test.shape[0] != y_test.shape[0]:
            raise RuntimeError(f"test X rows ({X_test.shape[0]}) != test labels length ({y_test.shape[0]})")
    else:
        test_adv_npy_path = Path(test_adv_npy_path)
        if not test_adv_npy_path.exists():
            raise FileNotFoundError(f"test_adv npy not found: {test_adv_npy_path}")
        X_clean = np.load(str(test_clean_npy_path), mmap_mode="r" if use_mmap else None).astype(np.float32)
        X_adv = np.load(str(test_adv_npy_path), mmap_mode="r" if use_mmap else None).astype(np.float32)
        X_test = np.vstack([X_clean, X_adv])
        if test_labels_npy_path is not None:
            y_test = np.load(str(test_labels_npy_path), mmap_mode="r" if use_mmap else None).astype(np.uint8)
            if y_test.shape[0] != X_test.shape[0]:
                raise RuntimeError(f"test labels length ({y_test.shape[0]}) != stacked test rows ({X_test.shape[0]})")
        else:
            y_test = np.concatenate([
                np.zeros(X_clean.shape[0], dtype=np.uint8),
                np.ones(X_adv.shape[0], dtype=np.uint8)
            ])

    pipeline, scaler, clf = _load_model(model_out_dir)
    y_pred, y_score = _predict_and_score(pipeline, scaler, clf, X_test)
    metrics = _compute_metrics(y_test, y_pred, y_score)

    print("TEST Eval results:")
    print(f"  acc={metrics['accuracy']:.4f}  prec={metrics['precision']:.4f}  recall={metrics['recall']:.4f}  f1={metrics['f1']:.4f}")
    if metrics["roc_auc"] is not None:
        print(f"  roc_auc={metrics['roc_auc']:.4f}")

    if csv_path is not None:
        csv_path = Path(csv_path)
        csv_path.parent.mkdir(parents=True, exist_ok=True)

        metrics_df = pd.DataFrame([metrics])
        metrics_df.to_csv(csv_path, index=False)

        if return_predictions:
            preds_path = csv_path.with_name(csv_path.stem + "_predictions.csv")
            preds_df = pd.DataFrame({
                "y_true": y_test,
                "y_pred": y_pred,
                "y_score": y_score
            })
            preds_df.to_csv(preds_path, index=False)
            print(f"Predictions saved to: {preds_path}")

        print(f"Metrics saved to: {csv_path}")

    return None



In [27]:
IMG_SIZES = [32, 64, 128, 256, 512, 1024]
ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
classifiers = ['LogisticRegression', 'RandomForest']
for IMG_SIZE in IMG_SIZES:
    print(f'Image Size: {IMG_SIZE} \n')
    for classifier in classifiers:
        print(f'Classifier: {classifier} \n')
        for ATTACK in ATTACKS:
            print (f'Training: {IMG_SIZE} - {ATTACK} - {classifier} \n')
            train_from_npy( train_npy_path = f"/kaggle/input/detectlid/Dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_raw.npy",
                            model_out_dir = f"./models/{ATTACK}-{IMG_SIZE}-{classifier}",
                            train_labels_npy_path=f"/kaggle/input/detectlid/Dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/train_labels.npy",
                            random_seed=42,
                            use_mmap=False,
                            classifier = classifier)
            print ('_________________________________\n')
        print ('==============================\n')
    print ('++++++++++++++++++++++++++++++++++\n')

Image Size: 32 

Classifier: LogisticRegression 

Training: 32 - fgsm - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


TRAIN Eval results:
  acc=0.5350  prec=0.5403  recall=0.4691  f1=0.5022
  roc_auc=0.5613
  confusion_matrix: [[292, 194], [258, 228]]
Saved pipeline -> models/fgsm-32-LogisticRegression/pipeline.joblib
Saved scaler -> models/fgsm-32-LogisticRegression/scaler.joblib
Saved classifier -> models/fgsm-32-LogisticRegression/clf.joblib
_________________________________

Training: 32 - bim - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


TRAIN Eval results:
  acc=0.6543  prec=0.7206  recall=0.5041  f1=0.5932
  roc_auc=0.7244
  confusion_matrix: [[391, 95], [241, 245]]
Saved pipeline -> models/bim-32-LogisticRegression/pipeline.joblib
Saved scaler -> models/bim-32-LogisticRegression/scaler.joblib
Saved classifier -> models/bim-32-LogisticRegression/clf.joblib
_________________________________

Training: 32 - pgd - LogisticRegression 

Using LogisticRegression as classifier
Fitting pipeline


...

TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/pgd-1024-RandomForest/pipeline.joblib
Saved scaler -> models/pgd-1024-RandomForest/scaler.joblib
Saved classifier -> models/pgd-1024-RandomForest/clf.joblib
_________________________________

Training: 1024 - df - RandomForest 

Using RandomForest as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/df-1024-RandomForest/pipeline.joblib
Saved scaler -> models/df-1024-RandomForest/scaler.joblib
Saved classifier -> models/df-1024-RandomForest/clf.joblib
_________________________________

Training: 1024 - cw - RandomForest 

Using RandomForest as classifier
Fitting pipeline


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


TRAIN Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[486, 0], [0, 486]]
Saved pipeline -> models/cw-1024-RandomForest/pipeline.joblib
Saved scaler -> models/cw-1024-RandomForest/scaler.joblib
Saved classifier -> models/cw-1024-RandomForest/clf.joblib
_________________________________


++++++++++++++++++++++++++++++++++



# Evaluate (Val Set)

In [28]:
IMG_SIZES = [32, 64, 128, 256, 512, 1024]
ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
classifiers = ['LogisticRegression', 'RandomForest']
for IMG_SIZE in IMG_SIZES:
    for ATTACK in ATTACKS:
        print (f'Attack Method: {ATTACK}\n')
        for classifier in classifiers:
            print (f'\n Classifier: {classifier}: \n')
            val_res = eval_val_from_npy(
                 val_npy_path = f"/kaggle/input/detectlid/Dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/val_raw.npy",
                 val_label_path = f"/kaggle/input/detectlid/Dataset/{IMG_SIZE}x{IMG_SIZE}/{ATTACK}/val_labels.npy",
                 model_out_dir = f"/kaggle/working/models/{ATTACK}-{IMG_SIZE}-{classifier}",
                 return_predictions=True,
                 use_mmap=False,
                 csv_path=f"./eval-VALSET/{ATTACK}-{DATASET}-{IMG_SIZE}-{classifier}_test_result.csv"
            )
            print("val metrics:", val_res["metrics"])
        print ('_____________________\n')

Attack Method: fgsm


 Classifier: LogisticRegression: 

VAL Eval results:
  acc=0.5432  prec=0.5574  recall=0.4198  f1=0.4789
  roc_auc=0.5856
  confusion_matrix: [[54, 27], [47, 34]]
Predictions saved to: eval-VALSET/fgsm-cw-32-LogisticRegression_test_result_predictions.csv
Metrics saved to: eval-VALSET/fgsm-cw-32-LogisticRegression_test_result.csv
val metrics: {'accuracy': 0.5432098765432098, 'precision': 0.5573770491803278, 'recall': 0.41975308641975306, 'f1': 0.4788732394366197, 'roc_auc': 0.5855814662399025, 'confusion_matrix': [[54, 27], [47, 34]], 'classification_report': {'0': {'precision': 0.5346534653465347, 'recall': 0.6666666666666666, 'f1-score': 0.5934065934065934, 'support': 81}, '1': {'precision': 0.5573770491803278, 'recall': 0.41975308641975306, 'f1-score': 0.4788732394366197, 'support': 81}, 'accuracy': 0.5432098765432098, 'macro avg': {'precision': 0.5460152572634313, 'recall': 0.5432098765432098, 'f1-score': 0.5361399164216065, 'support': 162}, 'weighted avg': {'p

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_job

VAL Eval results:
  acc=0.8457  prec=0.8333  recall=0.8642  f1=0.8485
  roc_auc=0.9123
  confusion_matrix: [[67, 14], [11, 70]]
Predictions saved to: eval-VALSET/bim-cw-32-RandomForest_test_result_predictions.csv
Metrics saved to: eval-VALSET/bim-cw-32-RandomForest_test_result.csv
val metrics: {'accuracy': 0.845679012345679, 'precision': 0.8333333333333334, 'recall': 0.8641975308641975, 'f1': 0.8484848484848484, 'roc_auc': 0.9122847126962352, 'confusion_matrix': [[67, 14], [11, 70]], 'classification_report': {'0': {'precision': 0.8589743589743589, 'recall': 0.8271604938271605, 'f1-score': 0.8427672955974842, 'support': 81}, '1': {'precision': 0.8333333333333334, 'recall': 0.8641975308641975, 'f1-score': 0.8484848484848484, 'support': 81}, 'accuracy': 0.845679012345679, 'macro avg': {'precision': 0.8461538461538461, 'recall': 0.845679012345679, 'f1-score': 0.8456260720411664, 'support': 162}, 'weighted avg': {'precision': 0.846153846153846, 'recall': 0.845679012345679, 'f1-score': 0.845

[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


VAL Eval results:
  acc=0.5247  prec=0.5270  recall=0.4815  f1=0.5032
  roc_auc=0.5374
  confusion_matrix: [[46, 35], [42, 39]]
Predictions saved to: eval-VALSET/df-cw-32-RandomForest_test_result_predictions.csv
Metrics saved to: eval-VALSET/df-cw-32-RandomForest_test_result.csv
val metrics: {'accuracy': 0.5246913580246914, 'precision': 0.527027027027027, 'recall': 0.48148148148148145, 'f1': 0.5032258064516129, 'roc_auc': 0.5374180765127267, 'confusion_matrix': [[46, 35], [42, 39]], 'classification_report': {'0': {'precision': 0.5227272727272727, 'recall': 0.5679012345679012, 'f1-score': 0.544378698224852, 'support': 81}, '1': {'precision': 0.527027027027027, 'recall': 0.48148148148148145, 'f1-score': 0.5032258064516129, 'support': 81}, 'accuracy': 0.5246913580246914, 'macro avg': {'precision': 0.5248771498771498, 'recall': 0.5246913580246914, 'f1-score': 0.5238022523382324, 'support': 162}, 'weighted avg': {'precision': 0.5248771498771498, 'recall': 0.5246913580246914, 'f1-score': 0.5

...

VAL Eval results:
  acc=0.8272  prec=0.8630  recall=0.7778  f1=0.8182
  roc_auc=0.9370
  confusion_matrix: [[71, 10], [18, 63]]
Predictions saved to: eval-VALSET/fgsm-cw-1024-RandomForest_test_result_predictions.csv
Metrics saved to: eval-VALSET/fgsm-cw-1024-RandomForest_test_result.csv
val metrics: {'accuracy': 0.8271604938271605, 'precision': 0.863013698630137, 'recall': 0.7777777777777778, 'f1': 0.8181818181818182, 'roc_auc': 0.9369760707209267, 'confusion_matrix': [[71, 10], [18, 63]], 'classification_report': {'0': {'precision': 0.797752808988764, 'recall': 0.8765432098765432, 'f1-score': 0.8352941176470587, 'support': 81}, '1': {'precision': 0.863013698630137, 'recall': 0.7777777777777778, 'f1-score': 0.8181818181818182, 'support': 81}, 'accuracy': 0.8271604938271605, 'macro avg': {'precision': 0.8303832538094504, 'recall': 0.8271604938271605, 'f1-score': 0.8267379679144384, 'support': 162}, 'weighted avg': {'precision': 0.8303832538094504, 'recall': 0.8271604938271605, 'f1-score

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


VAL Eval results:
  acc=1.0000  prec=1.0000  recall=1.0000  f1=1.0000
  roc_auc=1.0000
  confusion_matrix: [[81, 0], [0, 81]]
Predictions saved to: eval-VALSET/pgd-cw-1024-RandomForest_test_result_predictions.csv
Metrics saved to: eval-VALSET/pgd-cw-1024-RandomForest_test_result.csv
val metrics: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'roc_auc': 1.0, 'confusion_matrix': [[81, 0], [0, 81]], 'classification_report': {'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 81}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 81}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 162}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 162}}}
_____________________

Attack Method: df


 Classifier: LogisticRegression: 

VAL Eval results:
  acc=0.5617  prec=0.5490  recall=0.6914  f1=0.6120
  roc_auc=0.5574
  confusion_matrix: [[35, 46], [25, 56]]
Predictions saved to: eval

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


VAL Eval results:
  acc=0.4815  prec=0.4828  recall=0.5185  f1=0.5000
  roc_auc=0.5002
  confusion_matrix: [[36, 45], [39, 42]]
Predictions saved to: eval-VALSET/cw-cw-1024-RandomForest_test_result_predictions.csv
Metrics saved to: eval-VALSET/cw-cw-1024-RandomForest_test_result.csv
val metrics: {'accuracy': 0.48148148148148145, 'precision': 0.4827586206896552, 'recall': 0.5185185185185185, 'f1': 0.5, 'roc_auc': 0.5001524157902759, 'confusion_matrix': [[36, 45], [39, 42]], 'classification_report': {'0': {'precision': 0.48, 'recall': 0.4444444444444444, 'f1-score': 0.4615384615384615, 'support': 81}, '1': {'precision': 0.4827586206896552, 'recall': 0.5185185185185185, 'f1-score': 0.5, 'support': 81}, 'accuracy': 0.48148148148148145, 'macro avg': {'precision': 0.4813793103448276, 'recall': 0.48148148148148145, 'f1-score': 0.4807692307692307, 'support': 162}, 'weighted avg': {'precision': 0.4813793103448275, 'recall': 0.48148148148148145, 'f1-score': 0.4807692307692308, 'support': 162}}}


# Evaluate Dataset (Test Set)

In [29]:
classifier = 'LogisticRegression'
IMG_SIZES = [32, 64, 128, 256, 512, 1024]

ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
DATASETS = ['fgsm', 'bim', 'pgd', 'df', 'cw']

for IMG_SIZE in IMG_SIZES:
    for ATTACK in ATTACKS:
        for DATASET in DATASETS:
            print(f'Evaluate Against Test Set (Image Size: {IMG_SIZE}) \n Method: {ATTACK} \n Dataset: {DATASET} \n')
            eval_test_from_npy(
                test_clean_npy_path=f'/kaggle/input/detectlid/Dataset/{IMG_SIZE}x{IMG_SIZE}/{DATASET}/test_clean_raw.npy',
                test_adv_npy_path=f'/kaggle/input/detectlid/Dataset/{IMG_SIZE}x{IMG_SIZE}/{DATASET}/test_adv_raw.npy',
                model_out_dir=f"/kaggle/working/models/{ATTACK}-{IMG_SIZE}-{classifier}",
                test_labels_npy_path=None,
                return_predictions=False,
                use_mmap=False,
                csv_path=f"./eval/{ATTACK}-{DATASET}-{IMG_SIZE}-{classifier}_test_result.csv"
            )
            print ('_________________________\n')
    
        print ('===============================\n')




Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: fgsm 

TEST Eval results:
  acc=0.5610  prec=0.5658  recall=0.5244  f1=0.5443
  roc_auc=0.5709
Metrics saved to: eval/fgsm-fgsm-32-LogisticRegression_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: bim 

TEST Eval results:
  acc=0.4024  prec=0.3400  recall=0.2073  f1=0.2576
  roc_auc=0.3055
Metrics saved to: eval/fgsm-bim-32-LogisticRegression_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: pgd 

TEST Eval results:
  acc=0.4085  prec=0.3529  recall=0.2195  f1=0.2707
  roc_auc=0.3349
Metrics saved to: eval/fgsm-pgd-32-LogisticRegression_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: df 

TEST Eval results:
  acc=0.5061  prec=0.5075  recall=0.4146  f1=0.4564
  roc_auc=0.5070
Metrics saved to: eval/fgsm-df-32-LogisticRegression_tes

In [23]:
classifier = 'RandomForest'
IMG_SIZES = [32, 64, 128, 256, 512, 1024]

ATTACKS = ['fgsm', 'bim', 'pgd', 'df', 'cw']
DATASETS = ['fgsm', 'bim', 'pgd', 'df', 'cw']

for IMG_SIZE in IMG_SIZES:
    for ATTACK in ATTACKS:
        for DATASET in DATASETS:
            print(f'Evaluate Against Test Set (Image Size: {IMG_SIZE}) \n Method: {ATTACK} \n Dataset: {DATASET} \n')
            eval_test_from_npy(
                test_clean_npy_path=f'/kaggle/input/detectlid/Dataset/{IMG_SIZE}x{IMG_SIZE}/{DATASET}/test_clean_raw.npy',
                test_adv_npy_path=f'/kaggle/input/detectlid/Dataset/{IMG_SIZE}x{IMG_SIZE}/{DATASET}/test_adv_raw.npy',
                model_out_dir=f"/kaggle/working/models/{ATTACK}-{IMG_SIZE}-{classifier}",
                test_labels_npy_path=None,
                return_predictions=False,
                use_mmap=False,
                csv_path=f"./eval/{ATTACK}-{DATASET}-{IMG_SIZE}-{classifier}_test_result.csv"
            )
            print ('_________________________\n')
    
        print ('===============================\n')




Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: fgsm 

TEST Eval results:
  acc=0.5061  prec=0.5077  recall=0.4024  f1=0.4490
  roc_auc=0.5318
Metrics saved to: eval/fgsm-fgsm-32-RandomForest_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: bim 



[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_job

TEST Eval results:
  acc=0.5305  prec=0.5362  recall=0.4512  f1=0.4901
  roc_auc=0.5149
Metrics saved to: eval/fgsm-bim-32-RandomForest_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: pgd 

TEST Eval results:
  acc=0.5671  prec=0.5733  recall=0.5244  f1=0.5478
  roc_auc=0.5456
Metrics saved to: eval/fgsm-pgd-32-RandomForest_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: df 



[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


TEST Eval results:
  acc=0.4878  prec=0.4839  recall=0.3659  f1=0.4167
  roc_auc=0.4894
Metrics saved to: eval/fgsm-df-32-RandomForest_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 32) 
 Method: fgsm 
 Dataset: cw 

TEST Eval results:
  acc=0.4878  prec=0.4839  recall=0.3659  f1=0.4167
  roc_auc=0.5008
Metrics saved to: eval/fgsm-cw-32-RandomForest_test_result.csv
_________________________


Evaluate Against Test Set (Image Size: 32) 
 Method: bim 
 Dataset: fgsm 



...

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


TEST Eval results:
  acc=0.7134  prec=0.6699  recall=0.8415  f1=0.7459
  roc_auc=0.8028
Metrics saved to: eval/cw-bim-1024-RandomForest_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 1024) 
 Method: cw 
 Dataset: pgd 

TEST Eval results:
  acc=0.7439  prec=0.6852  recall=0.9024  f1=0.7789
  roc_auc=0.8020
Metrics saved to: eval/cw-pgd-1024-RandomForest_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 1024) 
 Method: cw 
 Dataset: df 



[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


TEST Eval results:
  acc=0.5488  prec=0.5526  recall=0.5122  f1=0.5316
  roc_auc=0.5296
Metrics saved to: eval/cw-df-1024-RandomForest_test_result.csv
_________________________

Evaluate Against Test Set (Image Size: 1024) 
 Method: cw 
 Dataset: cw 

TEST Eval results:
  acc=0.5000  prec=0.5000  recall=0.4146  f1=0.4533
  roc_auc=0.4965
Metrics saved to: eval/cw-cw-1024-RandomForest_test_result.csv
_________________________




[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [30]:
import hashlib
import os
from pathlib import Path
import zipfile
from datetime import datetime

# -------------------------------
# Config
# -------------------------------
input_dir = Path("/kaggle/working/eval")
output_zip = Path("/kaggle/working/evals-TEST.zip")
manifest_path = Path("/kaggle/working/manifest-evals-TEST-sha256.txt")
chunk_size = 8 * 1024 * 1024  # 8 MB
skip_files = {output_zip.name, manifest_path.name}

# -------------------------------
# Helper: compute SHA256
# -------------------------------
def sha256_of_file(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            h.update(chunk)
    return h.hexdigest()

# -------------------------------
# Collect files to zip
# -------------------------------
files_to_add = []
for root, dirs, files in os.walk(input_dir):
    for fname in files:
        full = Path(root) / fname
        rel = full.relative_to(input_dir)
        rel_str = str(rel).replace(os.sep, "/")
        if rel.name in skip_files:
            continue
        files_to_add.append((full, rel))

if not files_to_add:
    print(f"No files found in {input_dir} to archive.")
else:
    # -------------------------------
    # Create zip
    # -------------------------------
    print(f"Creating zip: {output_zip}")
    with zipfile.ZipFile(output_zip, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for full, rel in files_to_add:
            arcname = str(rel).replace(os.sep, "/")
            print(f"  adding: {arcname}")
            zf.write(full, arcname=arcname)

    # -------------------------------
    # Create manifest
    # -------------------------------
    print(f"Computing per-file SHA-256 and writing manifest: {manifest_path}")
    with manifest_path.open("w", encoding="utf-8") as mf:
        mf.write(f"# manifest generated: {datetime.utcnow().isoformat()}Z\n")
        mf.write(f"# input_dir: {input_dir}\n")
        mf.write("# format: <sha256>  <relative/path>\n")
        for full, rel in files_to_add:
            rel_unix = str(rel).replace(os.sep, "/")
            h = sha256_of_file(full)
            size = full.stat().st_size
            mf.write(f"{h}  {rel_unix}  # {size} bytes\n")

    # -------------------------------
    # Compute zip SHA256
    # -------------------------------
    zip_hash = sha256_of_file(output_zip)
    ziphash_path = output_zip.with_suffix(output_zip.suffix + ".sha256")
    with ziphash_path.open("w", encoding="utf-8") as zh:
        zh.write(f"{zip_hash}  {output_zip.name}\n")

    print("Done.")
    print(f"Created: {output_zip}")
    print(f"Created: {manifest_path}")
    print(f"Created: {ziphash_path}")


Creating zip: /kaggle/working/evals-TEST.zip
  adding: pgd-cw-128-RandomForest_test_result.csv
  adding: pgd-df-512-LogisticRegression_test_result.csv
  adding: df-bim-32-LogisticRegression_test_result.csv
  adding: pgd-cw-512-LogisticRegression_test_result.csv
  adding: df-cw-64-RandomForest_test_result.csv
  adding: df-pgd-128-LogisticRegression_test_result.csv
  adding: pgd-df-64-RandomForest_test_result.csv
  adding: df-pgd-1024-RandomForest_test_result.csv
  adding: fgsm-cw-1024-LogisticRegression_test_result.csv
  adding: df-df-256-LogisticRegression_test_result.csv
  adding: bim-fgsm-1024-LogisticRegression_test_result.csv
  adding: bim-fgsm-64-RandomForest_test_result.csv
  adding: df-df-512-LogisticRegression_test_result.csv
  adding: bim-df-256-LogisticRegression_test_result.csv
  adding: fgsm-df-64-LogisticRegression_test_result.csv
  adding: pgd-df-256-RandomForest_test_result.csv
  adding: pgd-fgsm-1024-RandomForest_test_result.csv
  adding: bim-df-64-RandomForest_test_res

In [31]:
import hashlib
import os
from pathlib import Path
import zipfile
from datetime import datetime

# -------------------------------
# Config
# -------------------------------
input_dir = Path("/kaggle/working/models")
output_zip = Path("/kaggle/working/models.zip")
manifest_path = Path("/kaggle/working/manifest-sha256.txt")
chunk_size = 8 * 1024 * 1024  # 8 MB
skip_files = {output_zip.name, manifest_path.name}

# -------------------------------
# Helper: compute SHA256
# -------------------------------
def sha256_of_file(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            h.update(chunk)
    return h.hexdigest()

# -------------------------------
# Collect files to zip
# -------------------------------
files_to_add = []
for root, dirs, files in os.walk(input_dir):
    for fname in files:
        full = Path(root) / fname
        rel = full.relative_to(input_dir)
        rel_str = str(rel).replace(os.sep, "/")
        if rel.name in skip_files:
            continue
        files_to_add.append((full, rel))

if not files_to_add:
    print(f"No files found in {input_dir} to archive.")
else:
    # -------------------------------
    # Create zip
    # -------------------------------
    print(f"Creating zip: {output_zip}")
    with zipfile.ZipFile(output_zip, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for full, rel in files_to_add:
            arcname = str(rel).replace(os.sep, "/")
            print(f"  adding: {arcname}")
            zf.write(full, arcname=arcname)

    # -------------------------------
    # Create manifest
    # -------------------------------
    print(f"Computing per-file SHA-256 and writing manifest: {manifest_path}")
    with manifest_path.open("w", encoding="utf-8") as mf:
        mf.write(f"# manifest generated: {datetime.utcnow().isoformat()}Z\n")
        mf.write(f"# input_dir: {input_dir}\n")
        mf.write("# format: <sha256>  <relative/path>\n")
        for full, rel in files_to_add:
            rel_unix = str(rel).replace(os.sep, "/")
            h = sha256_of_file(full)
            size = full.stat().st_size
            mf.write(f"{h}  {rel_unix}  # {size} bytes\n")

    # -------------------------------
    # Compute zip SHA256
    # -------------------------------
    zip_hash = sha256_of_file(output_zip)
    ziphash_path = output_zip.with_suffix(output_zip.suffix + ".sha256")
    with ziphash_path.open("w", encoding="utf-8") as zh:
        zh.write(f"{zip_hash}  {output_zip.name}\n")

    print("Done.")
    print(f"Created: {output_zip}")
    print(f"Created: {manifest_path}")
    print(f"Created: {ziphash_path}")


Creating zip: /kaggle/working/models.zip
  adding: fgsm-1024-RandomForest/scaler.joblib
  adding: fgsm-1024-RandomForest/train_meta.json
  adding: fgsm-1024-RandomForest/pipeline.joblib
  adding: fgsm-1024-RandomForest/clf.joblib
  adding: fgsm-256-LogisticRegression/scaler.joblib
  adding: fgsm-256-LogisticRegression/train_meta.json
  adding: fgsm-256-LogisticRegression/pipeline.joblib
  adding: fgsm-256-LogisticRegression/clf.joblib
  adding: df-512-RandomForest/scaler.joblib
  adding: df-512-RandomForest/train_meta.json
  adding: df-512-RandomForest/pipeline.joblib
  adding: df-512-RandomForest/clf.joblib
  adding: cw-256-RandomForest/scaler.joblib
  adding: cw-256-RandomForest/train_meta.json
  adding: cw-256-RandomForest/pipeline.joblib
  adding: cw-256-RandomForest/clf.joblib
  adding: pgd-64-RandomForest/scaler.joblib
  adding: pgd-64-RandomForest/train_meta.json
  adding: pgd-64-RandomForest/pipeline.joblib
  adding: pgd-64-RandomForest/clf.joblib
  adding: pgd-512-LogisticRegr

In [32]:
import hashlib
import os
from pathlib import Path
import zipfile
from datetime import datetime

# -------------------------------
# Config
# -------------------------------
input_dir = Path("/kaggle/working/eval-VALSET")
output_zip = Path("/kaggle/working/evals-VALSET.zip")
manifest_path = Path("/kaggle/working/manifest-evals-VALSET-sha256.txt")
chunk_size = 8 * 1024 * 1024  # 8 MB
skip_files = {output_zip.name, manifest_path.name}

# -------------------------------
# Helper: compute SHA256
# -------------------------------
def sha256_of_file(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            h.update(chunk)
    return h.hexdigest()

# -------------------------------
# Collect files to zip
# -------------------------------
files_to_add = []
for root, dirs, files in os.walk(input_dir):
    for fname in files:
        full = Path(root) / fname
        rel = full.relative_to(input_dir)
        rel_str = str(rel).replace(os.sep, "/")
        if rel.name in skip_files:
            continue
        files_to_add.append((full, rel))

if not files_to_add:
    print(f"No files found in {input_dir} to archive.")
else:
    # -------------------------------
    # Create zip
    # -------------------------------
    print(f"Creating zip: {output_zip}")
    with zipfile.ZipFile(output_zip, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for full, rel in files_to_add:
            arcname = str(rel).replace(os.sep, "/")
            print(f"  adding: {arcname}")
            zf.write(full, arcname=arcname)

    # -------------------------------
    # Create manifest
    # -------------------------------
    print(f"Computing per-file SHA-256 and writing manifest: {manifest_path}")
    with manifest_path.open("w", encoding="utf-8") as mf:
        mf.write(f"# manifest generated: {datetime.utcnow().isoformat()}Z\n")
        mf.write(f"# input_dir: {input_dir}\n")
        mf.write("# format: <sha256>  <relative/path>\n")
        for full, rel in files_to_add:
            rel_unix = str(rel).replace(os.sep, "/")
            h = sha256_of_file(full)
            size = full.stat().st_size
            mf.write(f"{h}  {rel_unix}  # {size} bytes\n")

    # -------------------------------
    # Compute zip SHA256
    # -------------------------------
    zip_hash = sha256_of_file(output_zip)
    ziphash_path = output_zip.with_suffix(output_zip.suffix + ".sha256")
    with ziphash_path.open("w", encoding="utf-8") as zh:
        zh.write(f"{zip_hash}  {output_zip.name}\n")

    print("Done.")
    print(f"Created: {output_zip}")
    print(f"Created: {manifest_path}")
    print(f"Created: {ziphash_path}")


Creating zip: /kaggle/working/evals-VALSET.zip
  adding: pgd-cw-128-RandomForest_test_result.csv
  adding: pgd-cw-512-LogisticRegression_test_result.csv
  adding: df-cw-64-RandomForest_test_result.csv
  adding: fgsm-cw-1024-LogisticRegression_test_result.csv
  adding: pgd-cw-512-LogisticRegression_test_result_predictions.csv
  adding: df-cw-128-LogisticRegression_test_result_predictions.csv
  adding: cw-cw-256-LogisticRegression_test_result_predictions.csv
  adding: fgsm-cw-64-RandomForest_test_result_predictions.csv
  adding: df-cw-512-LogisticRegression_test_result.csv
  adding: fgsm-cw-1024-RandomForest_test_result_predictions.csv
  adding: fgsm-cw-512-LogisticRegression_test_result.csv
  adding: bim-cw-128-LogisticRegression_test_result.csv
  adding: fgsm-cw-512-RandomForest_test_result.csv
  adding: bim-cw-32-LogisticRegression_test_result.csv
  adding: pgd-cw-64-LogisticRegression_test_result.csv
  adding: pgd-cw-512-RandomForest_test_result.csv
  adding: pgd-cw-32-RandomForest_t